In [1]:
!pip install vectorbt

In [2]:
import pandas as pd
from  helpers_db import get_engine, run_sql
import vectorbt as vbt
from sqlalchemy.sql import text

In [26]:
engine = get_engine()
df = None
with engine.begin() as conn:
  data = run_sql(conn, """SELECT dt as datetime, open AS Open, close AS Close, high AS High, low AS Low, volume AS Volume
                          FROM ticks_4h
                          WHERE symbol ='ADAUSD' AND dt >= '2022-01-01'::DATE
                          ORDER BY dt""")
  df = pd.DataFrame(data) #, columns=["datetime","Open","Close","High","Low","Volume"])

  df.rename(columns={"close": "Close", "open": "Open", "low": "Low", "high": "High", "volume": "Volume"}, inplace=True)
  df.set_index("datetime", inplace = True)
print(df)

                              Close      High       Low      Open  \
datetime                                                            
2022-01-02 00:00:00+00:00  1.355219  1.382888  1.350920  1.378900   
2022-01-02 04:00:00+00:00  1.355180  1.368837  1.342739  1.355850   
2022-01-02 08:00:00+00:00  1.363698  1.368100  1.348579  1.354782   
2022-01-02 12:00:00+00:00  1.369851  1.373986  1.354828  1.363694   
2022-01-02 16:00:00+00:00  1.362520  1.390039  1.356089  1.369755   
...                             ...       ...       ...       ...   
2024-04-13 17:00:00+00:00  0.451668  0.503959  0.439416  0.502804   
2024-04-13 21:00:00+00:00  0.445144  0.454869  0.400001  0.451605   
2024-04-14 01:00:00+00:00  0.446300  0.448764  0.433500  0.445726   
2024-04-14 09:00:00+00:00  0.456652  0.472072  0.446660  0.446660   
2024-04-14 13:00:00+00:00  0.466216  0.467358  0.456580  0.457425   

                                 Volume  
datetime                                 
2022-01-02 00:00:0

In [27]:
fast_ma = vbt.MA.run(df, 16, short_name='fast')
slow_ma = vbt.MA.run(df, 64, short_name='slow')

entries = fast_ma.ma_crossed_above(slow_ma)
entries

fast_window                   16                            
slow_window                   64                            
                           Close   High    Low   Open Volume
datetime                                                    
2022-01-02 00:00:00+00:00  False  False  False  False  False
2022-01-02 04:00:00+00:00  False  False  False  False  False
2022-01-02 08:00:00+00:00  False  False  False  False  False
2022-01-02 12:00:00+00:00  False  False  False  False  False
2022-01-02 16:00:00+00:00  False  False  False  False  False
...                          ...    ...    ...    ...    ...
2024-04-13 17:00:00+00:00  False  False  False  False  False
2024-04-13 21:00:00+00:00  False  False  False  False  False
2024-04-14 01:00:00+00:00  False  False  False  False  False
2024-04-14 09:00:00+00:00  False  False  False  False  False
2024-04-14 13:00:00+00:00  False  False  False  False  False

[5000 rows x 5 columns]

In [32]:
exits = fast_ma.ma_crossed_below(slow_ma)
exits

fast_window                   16                            
slow_window                   64                            
                           Close   High    Low   Open Volume
datetime                                                    
2022-01-02 00:00:00+00:00  False  False  False  False  False
2022-01-02 04:00:00+00:00  False  False  False  False  False
2022-01-02 08:00:00+00:00  False  False  False  False  False
2022-01-02 12:00:00+00:00  False  False  False  False  False
2022-01-02 16:00:00+00:00  False  False  False  False  False
...                          ...    ...    ...    ...    ...
2024-04-13 17:00:00+00:00  False  False  False  False  False
2024-04-13 21:00:00+00:00  False  False  False  False  False
2024-04-14 01:00:00+00:00  False  False  False  False  False
2024-04-14 09:00:00+00:00  False  False  False  False  False
2024-04-14 13:00:00+00:00  False  False  False  False  False

[5000 rows x 5 columns]

In [33]:
pf = vbt.Portfolio.from_signals(df, entries, exits)

pf.total_return()
# 0.636680693047752

fast_window  slow_window        
16           64           Close     0.121504
                          High     -0.256853
                          Low       0.101005
                          Open      0.114615
                          Volume   -0.999969
Name: total_return, dtype: float64

In [39]:
pf = vbt.PF.from_signals(
    data, 
    long_entries=entries, 
    long_exits=exits, 
)
pf.plot_trade_signals().show()

AttributeError: module 'vectorbt' has no attribute 'PF'

In [31]:
print(pf.stats())
print(pf.total_return())

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2022-01-02 00:00:00+00:00
End                           2024-04-14 13:00:00+00:00
Period                                             5000
Start Value                                       100.0
End Value                                     81.606044
Total Return [%]                             -18.393956
Benchmark Return [%]                         -66.032247
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              68.418651
Max Drawdown Duration                            4656.6
Total Trades                                       45.0
Total Closed Trades                                45.0
Total Open Trades                                   0.0
Open Trade PnL                                      0.0
Win Rate [%]                                   33.29163
Best Trade [%]                                98.226872
Worst Trade [%]                              -36

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_93811/2169671425.py:1: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x105726560>. Pass column to select a single column/group.

